## 17.1 의사결정나무란?
- 의사결정나무(decision tree): 다양한 의사결정 경로(decision path)와 결과(outcome)를 나타내는 데 나무 구조를 사용한다.

## 17.2. 엔트로피
- 엔트로피(entropy): 얼마만큼의 정보를 담고 있는가
- 엔트로피가 '무질서도(disorder)'를 의미한다는 말도 배운 적이 있을텐데 데이터의 '불확실성(uncertainty)'를 나타낼 때도 같은 표현을 사용한다.

In [4]:
from collections import defaultdict
#처음 키를 지정할 때 값을 주지 않으면 해당 키에 대한 값을 디폴트 값을 지정
#https://dongdongfather.tistory.com/69

int_dict = defaultdict(int) #자료형 지정 set도 가능
int_dict 
#설정값을 지정하지 않을 경우 키값은 0으로 지정

defaultdict(int, {})

In [8]:
print(int_dict['key'])
int_dict

0


defaultdict(int, {'key': 0})

In [9]:
int_dict['key2']='test'
int_dict

defaultdict(int, {'key': 0, 'key2': 'test'})

In [10]:
from collections import Counter, defaultdict

from functools import partial
import math, random

In [11]:
def entropy(class_probabilities):
    '''클래스에 속할 확률을 입력하면 엔트로피를 계산하라'''
    return sum(-p * math.log(p, 2)
              for p in class_probabilities if p)  # 확률이 0인 경우는 제외함

In [12]:
# 입력 데이터는 (input, label) 쌍으로 구성되어 있기 때문에, 각 클래스 레이블의 확률은 별도로 계산해 주어야 한다.
def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
           for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

## 17.3. 파티션의 엔트로피

In [13]:
def partition_entropy(subsets):
    '''subsets는 레이블이 있는 데이터의 list의 list이다.
    그에 대한 파티션 엔트로피를 구하라.'''
    total_count = sum(len(subset) for subset in subsets)
    
    return sum(data_entropy(subset) * len(subset) / total_count
              for subset in subsets)

## 17.4. 의사결정나무 만들기

In [14]:
inputs = [
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False)
    ]

In [15]:
# # 가장 낮은 엔트로피를 반환하는 파티션을 찾는다.
# def group_by(items, key_fn):
#     """returns a defaultdict(list), where each input item 
#     is in the list whose key is key_fn(item)"""
#     groups = defaultdict(list)
#     for item in items:
#         key = key_fn(item)
#         groups[key].append(item)
#     return groups
    
# def partition_by(inputs, attribute):
#     """returns a dict of inputs partitioned by the attribute
#     each input is a pair (attribute_dict, label)"""
#     return group_by(inputs, lambda x: x[0][attribute])    

In [ ]:
# 가장 낮은 엔트로피를 반환하는 파티션을 찾는다.
def partition_by(inputs, attribute):
    """attribute에 따라 inputs의 파티션을 나눈다"""
    groups = defaultdict(list)
    for input in inputs:
        key = input[0][attribute] #특정 attribute의 값을 불러오고
        groups[key].append(input) #이 input을 올바른 list에 추가 
    return groups

In [7]:
# 엔트로피를 계산
def partition_entropy_by(inputs, attribute):
    '''주어진 파티션에 대응되는 엔트로피를 계산'''
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

In [8]:
# 이제 전체 데이터셋에 대해 엔트로피를 최소화하는 파티션을 찾으면 된다.
for key in ['level', 'lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(inputs, key))

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617


In [9]:
senior_inputs = [(input, label)
                for input, label in inputs if input['level'] == 'Senior']

for key in ['lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(senior_inputs, key))

lang 0.4
tweets 0.0
phd 0.9509775004326938


## 17.5. 종합하기
> - True
> - False
> - (변수, 서브트리)로 구성된 tuple

- True는 어떤 값을 입력 받아도 True를 반환하는 잎 노드
- Fals는 어떤 값을 입력 받아도 False를 반환하는 잎 노드
- tuple은 변수와 그것으로 입력값을 분류한 서브트리로 구성된 결정 노드

In [19]:
def classify(tree, input):
    '''의사결정나무 tree로 주어진 입력값 input을 분류하자'''
    
    # 잎 노드이면 값을 반환하라
    if tree in [True, False]:
        return tree
    
    # 그게 아니라면 데이터의 변수로 파티션을 나누자
    # 키로 변수 값, 값으로 서브트리를 나타내는 dict를 사용하면 된다.
    attribute, subtree_dict = tree
    
    subtree_key = input.get(attribute)  # 만약 입력된 데이터의 변수 중 하나가 기존에 관찰되지 않았다면 None
    
    if subtree_key not in subtree_dict:  # 키에 해당하는 서브트리가 존재하지 않을 때
        subtree_key = None  # None 서브트리를 사용
    
    subtree = subtree_dict[subtree_key]  # 적절한 서브트리를 선택
    return classify(subtree, input)  # 그리고 입력된 데이터를 분류

In [23]:
def build_tree_id3(inputs, split_candidates=None):
    # 만약 파티션하는 첫 번째 단계라면
    # 입력된 데이터의 모든 변수가 파티션 기준 후보
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()
        
    # 입력된 데이터에서 True와 False의 개수를 세어 본다.
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues
    
    if num_trues == 0:  # True가 없다면 False 잎을 반환
        return False  
    
    if num_falses == 0:  # False가 없다면 True 잎을 반환
        return True
    
    if not split_candidates:  # 만약 파티션 기준으로 사용할 변수가 없다면
        return num_trues >= num_falses  # 다수결로 결과를 결정
    
    # 아니면 가장 적합한 변수를 기준으로 파티션
    best_attribute = min(split_candidates, key=partial(partition_entropy_by, inputs))
    
    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                     if a != best_attribute]
    
    # 재귀적으로 서브트리를 구축
    subtrees = { attribute_value : build_tree_id3(subset, new_candidates)
               for attribute_value, subset in partitions.items()}
    
    subtrees[None] = num_trues > num_falses  # 기본값
    
    return (best_attribute, subtrees)

In [24]:
print('building the tree')
tree = build_tree_id3(inputs)
print(tree)

print('Junior / Java / tweets / no phd', classify(tree,
                                                 {'level' : 'Junior',
                                                 'lang' : 'Java',
                                                 'tweets' : 'yes',
                                                 'phd' : 'no'}))

print('Junior / Java / tweets / phd', classify(tree,
                                               {'level' : 'Junior',
                                                'lang' : 'Java',
                                                'tweets' : 'yes',
                                                'phd' : 'yes'}))

print('Intern', classify(tree, {'level' : 'Intern'}))
print('Senior', classify(tree, {'level' : 'Senior'}))

building the tree
('level', {'Senior': ('tweets', {'no': False, 'yes': True, None: False}), 'Mid': True, 'Junior': ('phd', {'no': True, 'yes': False, None: True}), None: True})
Junior / Java / tweets / no phd True
Junior / Java / tweets / phd False
Intern True
Senior False


## 17.6. 랜덤포레스트
- 오버피팅을 방지할 수 있는 대표적인 방법 중 하나로 랜덤포레스트(random forests)가 있다. 한마디로 말하면 여러 개의 의사결정나무를 만들고, 그들의 다수결로 결과를 결정하는 방법이다.

In [22]:
def forest_classify(trees, input):
    votes = [classify(tree, input) for tree in trees]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]

In [ ]:
# 만약 파티션 기준으로 사용할 변수가 얼마 남지 않았다면 전부 사용하자
if len(split_candidates) <= self.num_split_candidates:
    sampled_split_candidates = split_candidates
    
# 아니라면 랜덤한 변수를 선택
else:
    sampled_split_candidates = random.sample(split_candidates, self.num_split_candidates)
    
# 선택한 변수 중에서 가장 적절한 변수를 선택
best_attribute = min(sampled_split_candidates, key=partial(partition_entropy_by, inputs))

partitions = partition_by(inputs, best_attribute)

## 17.7. 더 공부해 보고 싶다면
- scikit-learn에는 많은 의사결정나무 모델이 구현되어 있다. 게다가 ensemble 모듈이 있고, 이 안에 다른 앙상블 보델을 비롯해 RandomForestClassifier이 포함되어 있다.